# x402 Facilitator for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

In [72]:
import requests
import json
from datetime import datetime

# Facilitator endpoint - Single function with path-based routing
# FACILITATOR_URL = "https://facilitator.fretchen.eu"
FACILITATOR_URL = "http://localhost:8080"

VERIFY_URL = f"{FACILITATOR_URL}/verify"
SETTLE_URL = f"{FACILITATOR_URL}/settle"
SUPPORTED_URL = f"{FACILITATOR_URL}/supported"


print("🚀 x402 Facilitator Endpoints (Single Function with Path Routing):")
print(f"  Base: {FACILITATOR_URL}")
print(f"  Verify: {VERIFY_URL}")
print(f"  Settle: {SETTLE_URL}")
print(f"  Supported: {SUPPORTED_URL}")

🚀 x402 Facilitator Endpoints (Single Function with Path Routing):
  Base: http://localhost:8080
  Verify: http://localhost:8080/verify
  Settle: http://localhost:8080/settle
  Supported: http://localhost:8080/supported


## Test /supported Endpoint

The `/supported` endpoint returns information about which networks and payment schemes the facilitator supports.

**x402 v2 Changes:**
- Uses `x402Facilitator` class with registered schemes
- Returns `kinds` array with `{x402Version, scheme, network}` 
- Does NOT include asset details (USDC addresses, etc.) in response
- Asset information must be known by the client
- Custom `extensions` array shows additional capabilities (e.g., whitelist)

**Single Function with Path Routing:**
- Single Scaleway Function deployment
- Path routing: `/verify`, `/settle`, `/supported`
- x402 v2 standard compliant


In [73]:
# Test the /supported endpoint
response = requests.get(SUPPORTED_URL)

print(f"Status Code: {response.status_code}")
supported = response.json()
print(json.dumps(supported, indent=2))

print(f"\n✅ Supported Networks:")
for kind in supported['kinds']:
    print(f"  - {kind['network']} ({kind['scheme']} scheme, x402 v{kind['x402Version']})")
    # Note: x402 v2 does not include asset details in getSupported() response
    # Asset information (USDC addresses, etc.) must be known by the client

# Show custom extensions if present
if 'extensions' in supported and supported['extensions']:
    print(f"\n🔧 Custom Extensions:")
    for ext in supported['extensions']:
        print(f"  - {ext.get('name', 'unknown')}: {ext.get('description', '')}")


Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10"
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "abstract"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "abstract-testnet"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "base-sepolia"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "base"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "avalanche-fuji"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "avalanche"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "iotex"
    },
    {
      "x402Version": 1,
      "scheme": "exact",
      "network": "sei"
    },
    {
      "x402Version": 1,
      "scheme": "exact"

## Create EIP-712 Signature

In dieser Zelle kannst du zwischen **Testnet** (Optimism Sepolia) und **Mainnet** (Optimism mit echtem Geld) wechseln.

**🧪 Testnet (Standard):**
- Optimism Sepolia (Chain ID: 11155420)
- USDC: `0x5fd84259d66Cd46123540766Be93DFE6D43130D7`
- Kein echtes Geld - zum Testen gedacht

**💰 Mainnet (Echtes Geld):**
- Optimism Mainnet (Chain ID: 10)
- USDC: `0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85`
- ⚠️ **VORSICHT: Verwendet echtes USDC!**

Um zu Mainnet zu wechseln, setze `USE_MAINNET = True` in der nächsten Zelle.


In [74]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
# private_key = os.getenv("NFT_WALLET_PRIVATE_KEY")

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)

pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')
pay_to_address = Web3.to_checksum_address(pay_to_address)

# pay_to_address = "0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20"

print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
USE_MAINNET = False  # Set to True for Optimism Mainnet with REAL MONEY

if USE_MAINNET:
    CHAIN_ID = 10  # Optimism Mainnet
    USDC_ADDRESS = "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85"  # USDC on Optimism Mainnet
    USDC_NAME = "USD Coin"  # Mainnet uses "USD Coin"
    NETWORK_NAME = "Optimism Mainnet"
    CAIP2_NETWORK = "eip155:10"
    print(f"\n🚨 WARNING: Using REAL MONEY on {NETWORK_NAME}!")
else:
    CHAIN_ID = 11155420  # Optimism Sepolia
    USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"  # USDC on Optimism Sepolia
    USDC_NAME = "USDC"  # Testnet uses "USDC"
    NETWORK_NAME = "Optimism Sepolia (Testnet)"
    CAIP2_NETWORK = "eip155:11155420"
    print(f"\n🧪 Using testnet: {NETWORK_NAME}")

print(f"Chain ID: {CHAIN_ID}")
print(f"USDC Address: {USDC_ADDRESS}")
print(f"USDC Name: {USDC_NAME}")

# Payment configuration
payment_amount = "20000"  # $0.02 USDC (6 decimals) - amount for recipient

print(f"\n💰 Payment Details:")
print(f"  Payment amount: {payment_amount} ({int(payment_amount)/1e6:.2f} USDC)")

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60
valid_before = int(datetime.now().timestamp()) + 3600
nonce = "0x" + secrets.token_hex(32)

# EIP-712 domain for USDC on Optimism
# CRITICAL: Mainnet uses "USD Coin", Testnet uses "USDC"!
domain_data = {
    "name": USDC_NAME,  # Network-specific name
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 types for TransferWithAuthorization
types = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Message data - sign the payment amount
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(payment_amount),
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": nonce
}

# Create the full EIP-712 message
full_message = {
    "types": types,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# ✅ Use eth_keys for USDC-compatible signatures (manual EIP-712 hash)
# This is what USDC contract expects - raw signature over the EIP-712 hash
from eth_hash.auto import keccak

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\n🔐 Signature Details:")
print(f"Network: {NETWORK_NAME}")
print(f"EIP-712 Hash: {full_eip712_hash.hex()}")
print(f"Signature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"v: {v}, r: {r_bytes.hex()[:16]}..., s: {s_bytes.hex()[:16]}...")
print(f"Nonce: {nonce}")
print(f"\n✅ Authorized ${int(payment_amount)/1e6:.2f} USDC payment")
print(f"\n💡 Using correct domain name: '{USDC_NAME}' for {NETWORK_NAME}")


Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

🧪 Using testnet: Optimism Sepolia (Testnet)
Chain ID: 11155420
USDC Address: 0x5fd84259d66Cd46123540766Be93DFE6D43130D7
USDC Name: USDC

💰 Payment Details:
  Payment amount: 20000 (0.02 USDC)

🔐 Signature Details:
Network: Optimism Sepolia (Testnet)
EIP-712 Hash: 0b7c1cee2d6c9153e4453b78c8a56d8384d7d5c81c720d5c253b9271ede948fe
Signature: 0x58061a6bdf7eeecffc75fa85b0811fca3c510bd24012cf2c8a976b96862a9d3558d2affae4c2020ea659725808ee03dae8297feefab178fc4945553d4a3471071b
Signature length: 132 chars (should be 132 with 0x)
v: 27, r: 58061a6bdf7eeecf..., s: 58d2affae4c2020e...
Nonce: 0x3e6e776bdfa1daefe763d6844ead227de21b5687a7d9102ac1de5686027cc953

✅ Authorized $0.02 USDC payment

💡 Using correct domain name: 'USDC' for Optimism Sepolia (Testnet)


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

### Debug: Vergleiche EIP-712 Domain-Parameter

Prüfe, ob die Domain-Parameter (name, version) vom Contract mit unseren Annahmen übereinstimmen:

In [75]:
# Debug: Read actual EIP-712 domain parameters from the USDC contract
print(f"🔍 Debugging EIP-712 Domain Parameters for {NETWORK_NAME}")
print(f"Contract: {USDC_ADDRESS}\n")

# Try different methods to get domain info
methods_to_try = [
    # Method 1: ERC-5267 standard (eip712Domain)
    {
        "name": "eip712Domain",
        "abi": [{
            "inputs": [],
            "name": "eip712Domain",
            "outputs": [
                {"name": "fields", "type": "bytes1"},
                {"name": "name", "type": "string"},
                {"name": "version", "type": "string"},
                {"name": "chainId", "type": "uint256"},
                {"name": "verifyingContract", "type": "address"},
                {"name": "salt", "type": "bytes32"},
                {"name": "extensions", "type": "uint256[]"}
            ],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 2: Simple name() and version() getters
    {
        "name": "name",
        "abi": [{
            "inputs": [],
            "name": "name",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    {
        "name": "version",
        "abi": [{
            "inputs": [],
            "name": "version",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 3: DOMAIN_SEPARATOR (computed hash)
    {
        "name": "DOMAIN_SEPARATOR",
        "abi": [{
            "inputs": [],
            "name": "DOMAIN_SEPARATOR",
            "outputs": [{"name": "", "type": "bytes32"}],
            "stateMutability": "view",
            "type": "function"
        }]
    }
]

# Connect to network
if USE_MAINNET:
    rpc_url = 'https://mainnet.optimism.io'
else:
    rpc_url = 'https://sepolia.optimism.io'

w3 = Web3(Web3.HTTPProvider(rpc_url))

contract_info = {}

for method in methods_to_try:
    try:
        contract = w3.eth.contract(address=USDC_ADDRESS, abi=method["abi"])
        func = getattr(contract.functions, method["name"])
        result = func().call()
        contract_info[method["name"]] = result
        print(f"✅ {method['name']}(): {result}")
    except Exception as e:
        print(f"❌ {method['name']}(): {str(e)[:80]}...")

# Compare with our assumptions
print(f"\n📊 Comparison:")
print(f"Our assumptions:")
print(f"  name: '{domain_data['name']}'")
print(f"  version: '{domain_data['version']}'")
print(f"  chainId: {domain_data['chainId']}")
print(f"  verifyingContract: {domain_data['verifyingContract']}")

if "eip712Domain" in contract_info:
    fields, name, version, chainId, verifyingContract, salt, extensions = contract_info["eip712Domain"]
    print(f"\nContract's actual EIP-712 domain:")
    print(f"  name: '{name}'")
    print(f"  version: '{version}'")
    print(f"  chainId: {chainId}")
    print(f"  verifyingContract: {verifyingContract}")
    
    # Check for mismatches
    if name != domain_data['name']:
        print(f"\n⚠️  MISMATCH: name is '{name}' not '{domain_data['name']}'!")
    if version != domain_data['version']:
        print(f"⚠️  MISMATCH: version is '{version}' not '{domain_data['version']}'!")
    if chainId != domain_data['chainId']:
        print(f"⚠️  MISMATCH: chainId is {chainId} not {domain_data['chainId']}!")
else:
    if "name" in contract_info:
        print(f"\nContract's name: '{contract_info['name']}'")
        if contract_info['name'] != domain_data['name']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['name']}'")
    
    if "version" in contract_info:
        print(f"Contract's version: '{contract_info['version']}'")
        if contract_info['version'] != domain_data['version']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['version']}'")

if "DOMAIN_SEPARATOR" in contract_info:
    print(f"\n🔐 Contract's DOMAIN_SEPARATOR:")
    print(f"  {contract_info['DOMAIN_SEPARATOR'].hex()}")
    print(f"\nOur computed domain separator:")
    print(f"  {signable_message.header.hex()}")
    
    if contract_info['DOMAIN_SEPARATOR'] != signable_message.header:
        print(f"\n❌ DOMAIN_SEPARATOR MISMATCH! This explains the signature failure!")
    else:
        print(f"\n✅ Domain separators match!")

🔍 Debugging EIP-712 Domain Parameters for Optimism Sepolia (Testnet)
Contract: 0x5fd84259d66Cd46123540766Be93DFE6D43130D7

❌ eip712Domain(): ('execution reverted', '0x')...
✅ name(): USDC
✅ version(): 2
✅ DOMAIN_SEPARATOR(): b'\t\xd08\xa3\xe4`@\xfc7\xeb\x01\x17M\xbb\xcd\xb7\x98\x1f\xbd\x8e\xaf\xd9\xe1\xa8W\xb1\xc6x\x05\xdf\xb2\x9e'

📊 Comparison:
Our assumptions:
  name: 'USDC'
  version: '2'
  chainId: 11155420
  verifyingContract: 0x5fd84259d66Cd46123540766Be93DFE6D43130D7

Contract's name: 'USDC'
Contract's version: '2'

🔐 Contract's DOMAIN_SEPARATOR:
  09d038a3e46040fc37eb01174dbbcdb7981fbd8eafd9e1a857b1c67805dfb29e

Our computed domain separator:
  09d038a3e46040fc37eb01174dbbcdb7981fbd8eafd9e1a857b1c67805dfb29e

✅ Domain separators match!


In [76]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")
print(f"Network: {NETWORK_NAME}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to appropriate network
if USE_MAINNET:
    rpc_url = 'https://mainnet.optimism.io'
else:
    rpc_url = 'https://sepolia.optimism.io'

w3 = Web3(Web3.HTTPProvider(rpc_url))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")


Prüfe Nonce: 0x3e6e776bdfa1daefe763d6844ead227de21b5687a7d9102ac1de5686027cc953
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101
Network: Optimism Sepolia (Testnet)

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


In [85]:
# Test: Recover the signer from the signature and verify it matches
from eth_account.messages import _hash_eip191_message, encode_defunct
from eth_account import Account

print("🔐 Local Signature Verification Test")
print("=" * 80)

# The EIP-712 hash we signed
print(f"EIP-712 Hash: {full_eip712_hash.hex()}")
print(f"Signer (expected): {from_address}")
print(f"Signature: {signature_hex[:20]}...{signature_hex[-10:]}")

# Try to recover the address from the signature
try:
    # viem/ethers expect v as 27/28, but Account.recover_message expects the raw hash
    recovered = Account.recover_message(
        signable_message,
        signature=signature_hex
    )
    
    print(f"\n✅ Recovered address: {recovered}")
    
    if recovered.lower() == from_address.lower():
        print("✅ Signature is VALID! Address matches!")
    else:
        print(f"❌ Signature INVALID! Expected {from_address}, got {recovered}")
        
except Exception as e:
    print(f"\n❌ Signature recovery failed: {e}")
    print("\nTrying alternative recovery methods...")
    
    # Try with v-values 27 and 28
    for test_v in [27, 28]:
        try:
            test_sig = '0x' + r_bytes.hex() + s_bytes.hex() + format(test_v, '02x')
            recovered = Account.recover_message(signable_message, signature=test_sig)
            print(f"  v={test_v}: recovered {recovered} {'✅ MATCH!' if recovered.lower() == from_address.lower() else '❌'}")
        except Exception as e2:
            print(f"  v={test_v}: {str(e2)[:50]}...")


🔐 Local Signature Verification Test
EIP-712 Hash: 0b7c1cee2d6c9153e4453b78c8a56d8384d7d5c81c720d5c253b9271ede948fe
Signer (expected): 0x553179556FC2A39e535D65b921e01fA995E79101
Signature: 0x58061a6bdf7eeecffc...4a3471071b

✅ Recovered address: 0x553179556FC2A39e535D65b921e01fA995E79101
✅ Signature is VALID! Address matches!


In [ ]:
# Check if ALL addresses in our payload are properly checksummed
print("🔍 Address Checksum Verification")
print("=" * 80)

addresses_to_check = {
    "asset (USDC)": USDC_ADDRESS,
    "from (payer)": from_address,
    "to (recipient)": pay_to_address,
    "verifyingContract (in domain)": domain_data['verifyingContract']
}

all_checksummed = True
for name, addr in addresses_to_check.items():
    is_checksummed = Web3.is_checksum_address(addr)
    proper_checksum = Web3.to_checksum_address(addr)
    matches = addr == proper_checksum
    
    status = "✅" if matches else "❌"
    print(f"{status} {name}:")
    print(f"   Current: {addr}")
    if not matches:
        print(f"   Should be: {proper_checksum}")
        all_checksummed = False

print("\n" + "=" * 80)
if all_checksummed:
    print("✅ All addresses are properly checksummed!")
else:
    print("❌ Some addresses are NOT properly checksummed!")
    print("\n💡 This could cause EIP-712 hash mismatches!")
    print("   x402 v2 might checksum the addresses before hashing,")
    print("   but we signed with different casing!")


### 🔍 Address Checksum Test

EIP-712 ist case-sensitive! Wenn x402 v2 die Adressen anders formatiert, schlägt die Validierung fehl!

### 🔐 Test: Validiere Signatur lokal

Bevor wir zum Server senden, testen wir ob unsere Signatur überhaupt valide ist:

In [84]:
# Now create a valid x402 payment payload with our real signature

real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": CAIP2_NETWORK,  # Dynamic: eip155:10 (mainnet) or eip155:11155420 (testnet)
        "amount": payment_amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": USDC_NAME,  # CRITICAL: Must match what we signed with!
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": int(payment_amount),
            "validAfter": int(valid_after),
            "validBefore": int(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": CAIP2_NETWORK,  # Dynamic network
    "amount": payment_amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": USDC_NAME,  # CRITICAL: Must match what we signed with!
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

print(f"\n📤 Request Summary:")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment Amount: ${int(payment_amount)/1e6:.2f} USDC")
print(f"  Recipient: {pay_to_address}")

# Send to facilitator (deployed on Scaleway)
response = requests.post(
    VERIFY_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

result = response.json()
print(f"\n✅ Verify Response (Status {response.status_code}):")
print(json.dumps(result, indent=2))

if result.get('isValid'):
    print("\n🎉 Payment signature is valid and ready for settlement!")
    print(f"   Facilitator will transfer ${int(payment_amount)/1e6:.2f} to recipient")
else:
    invalid_reason = result.get('invalidReason', 'unknown')
    print(f"\n❌ Validation failed: {invalid_reason}")
    
    # If insufficient funds, check actual balance
    if 'insufficient' in invalid_reason.lower():
        print(f"\n💰 Checking payer's USDC balance...")
        
        # USDC ERC-20 ABI for balanceOf
        erc20_abi = [
            {
                "inputs": [{"name": "account", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "", "type": "uint256"}],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        # Connect to network
        if USE_MAINNET:
            rpc_url = 'https://mainnet.optimism.io'
        else:
            rpc_url = 'https://sepolia.optimism.io'
        
        w3 = Web3(Web3.HTTPProvider(rpc_url))
        usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
        
        # Check balance
        try:
            payer_balance = usdc_contract.functions.balanceOf(from_address).call()
            required_amount = int(payment_amount)
            
            print(f"\n📊 Balance Check:")
            print(f"   Payer address: {from_address}")
            print(f"   Current USDC balance: ${payer_balance/1e6:.6f} USDC")
            print(f"   Required for payment: ${required_amount/1e6:.6f} USDC")
            print(f"   Shortfall: ${(required_amount - payer_balance)/1e6:.6f} USDC")
            
            if payer_balance == 0:
                print(f"\n💡 Payer has NO USDC! Get USDC:")
                if USE_MAINNET:
                    print(f"   • Bridge: https://app.optimism.io/bridge")
                    print(f"   • Buy: https://www.coinbase.com/price/usd-coin")
                else:
                    print(f"   • Faucet: https://faucet.circle.com/")
                    print(f"   • Or bridge from Sepolia: https://app.optimism.io/bridge")
            else:
                print(f"\n💡 Payer needs more USDC:")
                print(f"   Send at least ${(required_amount - payer_balance)/1e6:.6f} USDC to:")
                print(f"   {from_address}")
                
        except Exception as e:
            print(f"\n⚠️  Could not check balance: {str(e)}")



📤 Request Summary:
  Network: Optimism Sepolia (Testnet)
  Payment Amount: $0.02 USDC
  Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

✅ Verify Response (Status 200):
{
  "isValid": false,
  "invalidReason": "invalid_exact_evm_payload_signature",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

❌ Validation failed: invalid_exact_evm_payload_signature


In [78]:
# 🔍 DEBUG: Compare what was SIGNED vs what is being SENT

print("=" * 80)
print("🔐 SIGNED VALUES (what signature was created for):")
print("=" * 80)
print(f"from:        {message_data['from']}")
print(f"to:          {message_data['to']}")
print(f"value:       {message_data['value']} (type: {type(message_data['value']).__name__})")
print(f"validAfter:  {message_data['validAfter']} (type: {type(message_data['validAfter']).__name__})")
print(f"validBefore: {message_data['validBefore']} (type: {type(message_data['validBefore']).__name__})")
print(f"nonce:       {message_data['nonce']}")

print("\n" + "=" * 80)
print("📤 PAYLOAD VALUES (what will be sent to facilitator):")
print("=" * 80)
auth = real_payment_payload['payload']['authorization']
print(f"from:        {auth['from']}")
print(f"to:          {auth['to']}")
print(f"value:       {auth['value']} (type: {type(auth['value']).__name__})")
print(f"validAfter:  {auth['validAfter']} (type: {type(auth['validAfter']).__name__})")
print(f"validBefore: {auth['validBefore']} (type: {type(auth['validBefore']).__name__})")
print(f"nonce:       {auth['nonce']}")

print("\n" + "=" * 80)
print("✅ MATCH CHECK:")
print("=" * 80)
matches = {
    "from": message_data['from'] == auth['from'],
    "to": message_data['to'] == auth['to'],
    "value": message_data['value'] == auth['value'],
    "validAfter": message_data['validAfter'] == auth['validAfter'],
    "validBefore": message_data['validBefore'] == auth['validBefore'],
    "nonce": message_data['nonce'] == auth['nonce']
}

for field, match in matches.items():
    status = "✅" if match else "❌"
    print(f"{status} {field}: {match}")

if not all(matches.values()):
    print("\n⚠️  MISMATCH DETECTED! Signature will be INVALID!")
    print("You must re-run the signature cell (Cell 6) to create a NEW signature!")
else:
    print("\n✅ All values match! Signature should be valid.")


🔐 SIGNED VALUES (what signature was created for):
from:        0x553179556FC2A39e535D65b921e01fA995E79101
to:          0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
value:       20000 (type: int)
validAfter:  1766406300 (type: int)
validBefore: 1766409960 (type: int)
nonce:       0x3e6e776bdfa1daefe763d6844ead227de21b5687a7d9102ac1de5686027cc953

📤 PAYLOAD VALUES (what will be sent to facilitator):
from:        0x553179556FC2A39e535D65b921e01fA995E79101
to:          0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
value:       20000 (type: int)
validAfter:  1766406300 (type: int)
validBefore: 1766409960 (type: int)
nonce:       0x3e6e776bdfa1daefe763d6844ead227de21b5687a7d9102ac1de5686027cc953

✅ MATCH CHECK:
✅ from: True
✅ to: True
✅ value: True
✅ validAfter: True
✅ validBefore: True
✅ nonce: True

✅ All values match! Signature should be valid.


In [79]:
# Check what actually gets sent as JSON
import json

print("🔬 JSON Serialization Test:")
print("=" * 80)

# Serialize the authorization object
auth_json = json.dumps(real_payment_payload['payload']['authorization'])
print("Authorization as JSON string:")
print(auth_json)
print()

# Parse it back to see what types we get
auth_parsed = json.loads(auth_json)
print("After JSON.parse (what the server receives):")
print(f"value:       {auth_parsed['value']} (type: {type(auth_parsed['value']).__name__})")
print(f"validAfter:  {auth_parsed['validAfter']} (type: {type(auth_parsed['validAfter']).__name__})")
print(f"validBefore: {auth_parsed['validBefore']} (type: {type(auth_parsed['validBefore']).__name__})")

print("\n" + "=" * 80)
print("⚠️  CRITICAL INSIGHT:")
print("=" * 80)

if isinstance(auth_parsed['value'], int):
    print("✅ JSON preserves integers - server should receive integers")
    print("   This is what x402 v2 expects for EIP-712 hash computation")
else:
    print("❌ JSON converts to different type!")
    print(f"   Server will receive: {type(auth_parsed['value']).__name__}")
    
print("\n💡 x402 v2 EVM reconstructs EIP-712 hash like this:")
print("   1. Takes authorization.value (expecting integer)")
print("   2. Encodes as uint256 for keccak256")
print("   3. If server receives string, hash will be wrong!")

🔬 JSON Serialization Test:
Authorization as JSON string:
{"from": "0x553179556FC2A39e535D65b921e01fA995E79101", "to": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C", "value": 20000, "validAfter": 1766406300, "validBefore": 1766409960, "nonce": "0x3e6e776bdfa1daefe763d6844ead227de21b5687a7d9102ac1de5686027cc953"}

After JSON.parse (what the server receives):
value:       20000 (type: int)
validAfter:  1766406300 (type: int)
validBefore: 1766409960 (type: int)

⚠️  CRITICAL INSIGHT:
✅ JSON preserves integers - server should receive integers
   This is what x402 v2 expects for EIP-712 hash computation

💡 x402 v2 EVM reconstructs EIP-712 hash like this:
   1. Takes authorization.value (expecting integer)
   2. Encodes as uint256 for keccak256
   3. If server receives string, hash will be wrong!


## 🔬 JSON Serialization Check

Das Problem könnte sein, wie Python die Werte als JSON serialisiert!

## 🐛 Debug: Check Signature vs Payload Match

**Critical Check:** Die Signatur muss EXAKT zum Payload passen!

Die häufigsten Probleme:
1. **Type Mismatch**: Signatur mit `int`, aber Payload sendet `str`
2. **Address Mismatch**: `from`/`to` Adressen stimmen nicht überein
3. **Value Mismatch**: `value` in Signatur ≠ `value` in Payload
4. **Nonce Reuse**: Nonce wurde bereits on-chain verwendet
5. **Domain Mismatch**: USDC contract name/version falsch


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [80]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to appropriate network
    if USE_MAINNET:
        rpc_url = 'https://mainnet.optimism.io'
    else:
        rpc_url = 'https://sepolia.optimism.io'
    
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"Network: {NETWORK_NAME}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            if USE_MAINNET:
                print(f"Bridge: https://app.optimism.io/bridge")
            else:
                print(f"Faucet: https://app.optimism.io/faucet")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print(f"❌ Konnte nicht mit {NETWORK_NAME} verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")


Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
Network: Optimism Sepolia (Testnet)
ETH Balance: 0.049999061348277237 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [81]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

print(f"\n💸 Attempting Settlement...")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment: ${int(payment_amount)/1e6:.2f} USDC → {pay_to_address}")

if USE_MAINNET:
    print(f"\n🚨 WARNING: This will execute a REAL transaction with REAL MONEY!")
    confirm = input("Type 'YES' to confirm: ")
    if confirm != 'YES':
        print("❌ Settlement cancelled")
        raise Exception("User cancelled mainnet settlement")

settle_response = requests.post(
    SETTLE_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

settle_result = settle_response.json()
print(f"\n📦 Settle Response (Status {settle_response.status_code}):")
print(json.dumps(settle_result, indent=2))

if settle_result.get('success'):
    tx_hash = settle_result.get('transaction')  # API returns 'transaction', not 'transactionHash'
    print(f"\n🎉 Settlement erfolgreich!")
    print(f"\n📝 Transaction Details:")
    print(f"   Hash: {tx_hash}")
    print(f"   Recipient received: ${int(payment_amount)/1e6:.2f} USDC")
else:
    tx_hash = None
    print(f"\n❌ Settlement fehlgeschlagen")
    error_reason = settle_result.get('errorReason', settle_result.get('invalidReason', 'unknown'))
    print(f"   Grund: {error_reason}")
    
    if 'insufficient' in error_reason.lower():
        print(f"\n💡 Tipp: Stelle sicher, dass:")
        print(f"   - Payer hat ${int(payment_amount)/1e6:.2f} USDC auf {NETWORK_NAME}")
        print(f"   - Facilitator hat ETH für Gas")



💸 Attempting Settlement...
  Network: Optimism Sepolia (Testnet)
  Payment: $0.02 USDC → 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

📦 Settle Response (Status 200):
{
  "success": false,
  "errorReason": "invalid_exact_evm_payload_signature",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "",
  "network": "eip155:11155420"
}

❌ Settlement fehlgeschlagen
   Grund: invalid_exact_evm_payload_signature


In [82]:
# Display transaction on block explorer
if tx_hash:
    # Block explorer URL depends on network
    if USE_MAINNET:
        explorer_url = f"https://optimistic.etherscan.io/tx/{tx_hash}"
    else:
        explorer_url = f"https://sepolia-optimism.etherscan.io/tx/{tx_hash}"
    
    print(f"🔍 Block Explorer:")
    print(f"   {explorer_url}")
    print(f"\n📊 View transaction details:")
    print(f"   • Transaction hash: {tx_hash}")
    print(f"   • Network: {NETWORK_NAME} ({CAIP2_NETWORK})")
    print(f"   • Token: USDC at {USDC_ADDRESS}")
else:
    print(f"\n⚠️  No transaction hash available")


⚠️  No transaction hash available


In [83]:
# Check USDC balances to verify the transfer actually happened
if tx_hash:
    print(f"💰 Checking USDC Balances on {NETWORK_NAME}...\n")
    
    # USDC ERC-20 ABI for balanceOf
    erc20_abi = [
        {
            "inputs": [{"name": "account", "type": "address"}],
            "name": "balanceOf",
            "outputs": [{"name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function"
        }
    ]
    
    # Connect to network
    if USE_MAINNET:
        rpc_url = 'https://mainnet.optimism.io'
    else:
        rpc_url = 'https://sepolia.optimism.io'
    
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
    
    # Check balances
    payer_balance = usdc_contract.functions.balanceOf(from_address).call()
    recipient_balance = usdc_contract.functions.balanceOf(pay_to_address).call()
    facilitator_balance = usdc_contract.functions.balanceOf(facilitator_address).call()
    
    print(f"📊 USDC Balances:")
    print(f"   Payer ({from_address[:10]}...): ${payer_balance/1e6:.6f} USDC")
    print(f"   Recipient ({pay_to_address[:10]}...): ${recipient_balance/1e6:.6f} USDC")
    print(f"   Facilitator ({facilitator_address[:10]}...): ${facilitator_balance/1e6:.6f} USDC")
    
    print(f"\n💡 Warum erscheint die Transaktion nicht in MetaMask Activity?")
    print(f"   • TransferWithAuthorization wird vom Facilitator ausgeführt")
    print(f"   • MetaMask zeigt nur selbst gesendete Transaktionen")
    print(f"   • Die USDC wurden trotzdem korrekt transferiert!")
    print(f"   • Prüfe den Token-Balance in MetaMask (nicht die Activity)")
    print(f"\n🔍 So siehst du den Transfer:")
    print(f"   1. Öffne Etherscan: {explorer_url}")
    print(f"   2. Klicke auf 'Logs' Tab um die Transfer Events zu sehen")
    print(f"   3. Oder gehe zu Token Transfers auf der Payer/Recipient Adresse")
else:
    print(f"\n⚠️  No transaction hash available")


⚠️  No transaction hash available
